# CFG

In [1]:
# ====================================================
# CFG
# ====================================================
class CFG:
    num_workers=4
    path="/kaggle/input/pppm-deberta-v3-large-baseline-w-w-b-t/"
#     path="../input/pppm-bert-for-patents-baseline-w-w-b-train/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
#     model="anferico/bert-for-patents"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=60 # CHANGED: 133>60
    seed=42
    n_fold=4
    trn_fold=[0]
    
INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'

# Library

In [2]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
# import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import itertools
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


torch.__version__: 1.9.1
Found existing installation: transformers 4.16.2
Uninstalling transformers-4.16.2:
  Successfully uninstalled transformers-4.16.2


Found existing installation: tokenizers 0.11.6
Uninstalling tokenizers-0.11.6:
  Successfully uninstalled tokenizers-0.11.6


Looking in links: ../input/pppm-pip-wheels-dataset
Processing /kaggle/input/pppm-pip-wheels-dataset/transformers-4.16.2-py3-none-any.whl
Processing /kaggle/input/pppm-pip-wheels-dataset/tokenizers-0.11.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


Looking in links: ../input/pppm-pip-wheels-dataset


tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


# Utils

In [3]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# tokenizer

In [4]:
# ====================================================
# tokenizer
# ====================================================
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')

# Dataset

In [5]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

# Model

In [6]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

# inference

In [7]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
#     tk0 = tqdm(test_loader, total=len(test_loader))
#     for inputs in tk0:
    for inputs in test_loader:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

# Quora (Inference Pipeline)

In [8]:
pip install -q keybert

Note: you may need to restart the kernel to use updated packages.


In [9]:
from keybert import KeyBERT
kw_model = KeyBERT()

def get_keywords(data, n_top=5, n_words=2):
    return kw_model.extract_keywords(
        data, 
        keyphrase_ngram_range=(1, n_words), 
        # use_mmr=True, nr_candidates=20, # diversity
        top_n=n_top,
        # highlight=True
    )

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [10]:
!mkdir -p data
!cp ../input/question-pairs-dataset/questions.csv ./data

In [11]:
N_SAMPLES = 20000

quora_df = pd.read_csv('./data/questions.csv').fillna("")
quora_df = quora_df.head(N_SAMPLES)
quora_df = quora_df.drop(columns=['qid1', 'qid2', 'is_duplicate'])
quora_df.head()

,id,question1,question2
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [12]:
pip install -q -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [13]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


def get_score_sbert(sent1, sent2):
    embedding_1= model.encode(sent1, convert_to_tensor=True, show_progress_bar=False)
    embedding_2 = model.encode(sent2, convert_to_tensor=True, show_progress_bar=False)
    
    return util.pytorch_cos_sim(embedding_1, embedding_2)[0].item()

get_score_sbert("I'm happy", "I'm full of happiness")

0.6002566814422607

In [14]:
%%time
quora_df['label'] = quora_df.apply(lambda x: get_score_sbert(x['question1'], x['question2']), axis=1)
quora_df.to_csv('quora_labels.csv', index=False)
quora_df.head()

CPU times: user 3min 51s, sys: 196 ms, total: 3min 51s
Wall time: 3min 51s


,id,question1,question2,label
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.912277
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.655141
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.515561
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.104022
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.325348


In [15]:
%%time
# Keyword extraction -> keyphrase pairs dataframe
N_TOP_KEYWORD = 5

test = pd.DataFrame(columns=['anchor', 'target', 'sample_id', 'text'])

for i in tqdm(range(len(quora_df)), total=len(quora_df)):
    sample = quora_df.iloc[i]
    k1list = [key for key, _ in get_keywords(sample.question1, n_top=N_TOP_KEYWORD)]
    k2list = [key for key, _ in get_keywords(sample.question2, n_top=N_TOP_KEYWORD)]
    keyword_pairs = itertools.product(k1list, k2list)
    df = pd.DataFrame(keyword_pairs, columns=['anchor', 'target'])
    df['sample_id'] = sample.id
    df['sample_id'] = df['sample_id'].astype(int)
    test = pd.concat([test, df], ignore_index=True)    

test['text'] = test['anchor'] + '[SEP]' + test['target']
test.head()

  0%|          | 0/20000 [00:00<?, ?it/s]

CPU times: user 2h 3min 10s, sys: 1min 32s, total: 2h 4min 43s
Wall time: 2h 3min 4s


,anchor,target,sample_id,text
0,invest share,invest share,0,invest share[SEP]invest share
1,invest share,share market,0,invest share[SEP]share market
2,invest share,guide invest,0,invest share[SEP]guide invest
3,invest share,invest,0,invest share[SEP]invest
4,invest share,step guide,0,invest share[SEP]step guide


In [16]:
%%time
# Keyphrase similarity prediction -> Keyphrase pairs + scores

test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []

for fold in CFG.trn_fold:
    print(fold)
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
#     print(prediction)
    
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()

predictions = np.mean(predictions, axis=0)
test['score'] = predictions
test.head()

0
CPU times: user 1h 6min 25s, sys: 14.5 s, total: 1h 6min 40s
Wall time: 1h 6min 53s


,anchor,target,sample_id,text,score
0,invest share,invest share,0,invest share[SEP]invest share,0.999466
1,invest share,share market,0,invest share[SEP]share market,0.000559
2,invest share,guide invest,0,invest share[SEP]guide invest,0.281476
3,invest share,invest,0,invest share[SEP]invest,0.487118
4,invest share,step guide,0,invest share[SEP]step guide,0.000686


In [17]:
test.to_csv('/kaggle/working/keyphrase_scores.csv', index=False)

In [18]:
# Keyphrase pairs + scores -> Question pairs + scores
TOP_N_METRIC = 5

results = quora_df
results[f'top_{TOP_N_METRIC}_mean'] = test.groupby('sample_id')['score'].apply(lambda x: x.nlargest(TOP_N_METRIC).mean())
results.to_csv('quora_results.csv', index=False)
abs_diff = (results[f'top_{TOP_N_METRIC}_mean'] - results['label']).abs()
mean_abs_diff = abs_diff.mean()
print(f'Mean difference: {mean_abs_diff:.3f}')
results.head()

Mean difference: 0.128


,id,question1,question2,label,top_5_mean
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.912277,0.896803
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.655141,0.753013
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.515561,0.626512
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.104022,0.061979
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.325348,0.345740


# Post Process with MLP

In [19]:
from tqdm import tqdm

all_ids = test.sample_id.unique()

inputs = []
labels = []
for sid in tqdm(all_ids):
    scores = test[test.sample_id == sid]['score'].values
    if len(scores) < 25:
        scores = np.pad(scores, (0, 25-len(scores)), 'constant')
    
    inputs.append(scores)
    labels.append(results[results.id == sid].label.values[0])
    
inputs = np.stack(inputs)
labels = np.array(labels)

inputs.shape, labels.shape

100%|██████████| 19980/19980 [09:39<00:00, 34.45it/s]


((19980, 25), (19980,))

In [20]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs, labels, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14985, 25), (4995, 25), (14985,), (4995,))

In [21]:
%%time
regr = MLPRegressor(random_state=42, max_iter=1000)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

CPU times: user 4.15 s, sys: 1.63 s, total: 5.78 s
Wall time: 2.9 s


0.7479428822820278

In [22]:
preds = regr.predict(X_test)

In [23]:
import pickle

filename = 'regressor_mlp.sav'
pickle.dump(regr, open(filename, 'wb'))
regr2 = pickle.load(open("regressor_mlp.sav", 'rb'))

In [24]:
def get_mlp_score(row):
    sid = row['id']
    scores = test[test.sample_id == sid]['score'].values
    if len(scores) < 25:
        scores = np.pad(scores, (0, 25-len(scores)), 'constant')
        
    return regr2.predict([scores])[0]

results[f'mlp'] = results.apply(lambda row: get_mlp_score(row), axis=1)
results.head()

,id,question1,question2,label,top_5_mean,mlp
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.912277,0.896803,0.840787
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.655141,0.753013,0.626862
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.515561,0.626512,0.644954
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.104022,0.061979,0.111782
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.325348,0.345740,0.349320


In [25]:
abs_diff = (results['mlp'] - results['label']).abs()
mean_abs_diff = abs_diff.mean()
print(f'Mean difference: {mean_abs_diff:.3f}')

Mean difference: 0.100


# Test

In [26]:
sample = quora_df.iloc[0]

q1 = sample.question1
q2 = sample.question2
print(f"q1: {q1}\nq2: {q2}")

k1, k2 = get_keywords(q1, 5), get_keywords(q2, 5)
k1, k2

q1: What is the step by step guide to invest in share market in india?
q2: What is the step by step guide to invest in share market?


([('invest share', 0.6415),
  ('share market', 0.6207),
  ('guide invest', 0.5644),
  ('market india', 0.5642),
  ('invest', 0.5359)],
 [('invest share', 0.6814),
  ('share market', 0.6392),
  ('guide invest', 0.6103),
  ('invest', 0.5822),
  ('step guide', 0.4547)])